In [11]:
import os
import json
import csv
import numpy as np

import pandas as pd

In [12]:
### dataset info and constants
data_year=2020
file_dir = "data/china_isd_lite_" + str(data_year)
print(file_dir)
# list head
year = 0
month = 1
day = 2
hour = 3
air_temperature = 4
dew_point_temperature = 5
sea_level_pressure = 6
wind_direction = 7
wind_speed_rate = 8
sky_condition_total_coverage_code = 9
### sky_condition_total_coverage_code
## DOMAIN:
#  0: None, SKC or CLR
#  1: One okta - 1/10 or less but not zero
#  2: Two oktas - 2/10 - 3/10, or FEW
#  3: Three oktas - 4/10
#  4: Four oktas - 5/10, or SCT
#  5: Five oktas - 6/10
#  6: Six oktas - 7/10 - 8/10
#  7: Seven oktas - 9/10 or more but not 10/10, or BKN
#  8: Eight oktas - 10/10, or OVC
#  9: Sky obscured, or cloud amount cannot be estimated
# 10: Partial obscuration
# 11: Thin scattered
# 12: Scattered
# 13: Dark scattered
# 14: Thin broken
# 15: Broken
# 16: Dark broken
# 17: Thin overcast
# 18: Overcast
# 19: Dark overcast
liquid_precipitation_pepth_dimension_1_hour = 10
liquid_precipitation_pepth_dimension_6_hour = 11

MISSING_VALUE = "-9999"

data/china_isd_lite_2020


In [7]:
selected_area = air_temperature

In [ ]:
# step 1: read total stations' file

total_stations = {}
count_missing_total = 0
for file_name in os.listdir(file_dir):
  # obtain site_id by file_name
  station_id = file_name.split("-")[0]

  file_object = open(file_dir + "/"+file_name, 'r')
  station_val = {}
  # station_missing_index = []
  # station_missing_count = 0
  # line_index = 0


  while(True):
    line = file_object.readline().split()
    if (len(line) == 12):
      # only save valid value  { "${month}${day}${hour}": value }
      key = line[month] + line[day] + line[hour]
      if (line[selected_area]==MISSING_VALUE):
        ...
        # count_missing_total += 1
        # station_missing_count+=1
        # station_missing_index.append(line_index)
      else:
        station_val[key] = int(line[selected_area])
    elif (len(line) == 0):
      break
    # line_index+=1
  # print(station_val)
  total_stations[station_id] = station_val
  # {
  #     "val": station_val,
  #   # "missing_count": station_missing_count,
  #   # "missing_index": station_missing_index,
  # }

  # next line output saved in missing count.txt
  # print(station_id, station_missing_count)

# print(count_missing_total)
# out_file = open("step1.json", "w")
# json.dump(total_stations, out_file)
# out_file.close()

### statistics
# 415 sites

## air_temperature
# missing_value_count: 2306
# each site missing saved in missing count.txt

## liquid_precipitation_pepth_dimension_1_hour
# missing_value_count: 1358702
# each site missing saved in water.txt


In [38]:
# filter and fulfil missing value
# month

import calendar

fixed_data = {}

for id in total_stations:
  count = 0
  series = pd.Series([], dtype=pd.StringDtype())
  for month in range(12):
    m = str(month + 1).zfill(2)
    monthrange = calendar.monthrange(data_year, month+1)
    # day
    for day in range(monthrange[1]):
      d = str(day + 1).zfill(2)
      # hour
      for hour_count in range(8):
        hour = hour_count*3
        h = str(hour).zfill(2)

        key = m+d+h
        # print(key)

        if (key in total_stations[id]):
          series[key] = total_stations[id][key]
        else:
          series[key] = np.nan
          count +=1
  fixed_data[id] = series
  # print(series)
  print(id, count)
  # print(series)


450070 6
450110 14
450320 25
450350 37
450390 25
450440 70
450450 1555
466860 16
466960 16
467400 17
467403 1095
467430 71
467700 60
501360 15
503530 10
504340 11
504680 15
505270 2515
505480 379
505570 15
505640 378
506030 13
506320 9
506580 377
507270 22
507450 11
507560 17
507740 12
507880 10
508440 2560
508540 376
508880 376
509150 10
509490 26
509530 15
509630 10
509680 381
509780 10
509830 379
510530 377
510760 10
510870 11
511330 14
511560 11
512430 9
512880 10
513340 388
513790 375
514310 12
514630 8
514635 17
514670 379
514950 379
515420 380
515730 7
516440 12
516560 12
517090 6
517110 375
517160 9
517300 376
517470 21
517650 19
517770 12
518110 10
518180 381
518280 9
518860 10
521120 376
522030 10
522670 27
523180 2927
523230 13
523780 378
524180 10
524360 376
524950 9
525330 20
525331 19
526020 18
526520 8
526810 10
527130 10
527370 375
527540 12
527870 376
528180 12
528360 15
528660 14
529080 393
529550 24
529830 10
529960 376
530680 11
530830 376
531490 24
531920 18
532310

In [121]:
def interpolate(series):
  new_series = series
  for i in range(len(series)):
    if (series[i]=='nan'):
      step = 1
      before_count = 0
      after_count = 0
      before_val_accumulate = 0
      after_val_accumulate = 0
      while step<5:
        before_index = i-8*step
        after_index = i+8*step
        if (before_index >= 0) & (after_index < len(series)):
          if (series[before_index] != 'nan'):
            before_count+=1
            before_val_accumulate += series[before_index]

          if (series[after_index] != 'nan'):
            after_count += 1
            after_val_accumulate += series[after_index]
        else: break
        step += 1

      if ((before_count==0)|(after_count==0)):
        print('failed', i)
      else:
         new_series[i] = (before_val_accumulate/before_count +
                        after_val_accumulate/after_count)/2
  print(len(series))
  return new_series


In [123]:
# drop stations
# drop_list = ['450450',
#              '467403',
#              '505270',
#              '508440',
#              '523180',
#              '536730',
#              '544360',
#              '545340',
#              '548525',
#              '564620',
#              '564625',
#              '568380',
#              '570140',
#              '580280',
#              '586590',
#              '591625',
#              '597580',
#              '538660',
#              '542730',
#              '570250',
#              '597920',
#              '599970']
count = 0
air_data = {}
# for id in drop_list:
#   fixed_data.pop(id)
for id in fixed_data:
  print(id)
  m = fixed_data[id].interpolate(
      limit_direction='both', method='linear', limit=1)
  list = m.fillna('nan').tolist()
  list = [int(i) for i in interpolate(list)]
  air_data[id] = list


out_file = open("air_data.json", "w")
json.dump(air_data, out_file)
out_file.close()


450070
2928
450110
2928
450320
2928
450350
2928
450390
2928
450440
2928
466860
2928
466960
2928
467400
2928
467430
2928
467700
2928
501360
2928
503530
2928
504340
2928
504680
2928
505480
2928
505570
2928
505640
2928
506030
2928
506320
2928
506580
2928
507270
2928
507450
2928
507560
2928
507740
2928
507880
2928
508540
2928
508880
2928
509150
2928
509490
2928
509530
2928
509630
2928
509680
2928
509780
2928
509830
2928
510530
2928
510760
2928
510870
2928
511330
2928
511560
2928
512430
2928
512880
2928
513340
2928
513790
2928
514310
2928
514630
2928
514635
2928
514670
2928
514950
2928
515420
2928
515730
2928
516440
2928
516560
2928
517090
2928
517110
2928
517160
2928
517300
2928
517470
2928
517650
2928
517770
2928
518110
2928
518180
2928
518280
2928
518860
2928
521120
2928
522030
2928
522670
2928
523230
2928
523780
2928
524180
2928
524360
2928
524950
2928
525330
2928
525331
2928
526020
2928
526520
2928
526810
2928
527130
2928
527370
2928
527540
2928
527870
2928
528180
2928
528360
2928
5286

In [119]:
# step 2: read stations' locations,  site_id-> [lat, lon] and station name

df=pd.read_csv('isd-history.csv')
df2 = df[['USAF', 'STATION NAME', 'LAT', 'LON']]

staion_info_dict = {}
valid_count = 0
total_count=0
for i, j in df2.iterrows():
  total_count+=1
  valid = (not pd.isnull(j['STATION NAME'])) & (
      not pd.isnull(j['LON'])) & (not pd.isnull(j['LAT']))
  if (valid):
    valid_count+=1
    staion_info_dict[j['USAF']]={
        'name': j['STATION NAME'],
        'lon': j['LON'],
        'lat': j['LAT']
    }

print(total_count, valid_count)
# out_file = open("step2.json", "w")
# json.dump(staion_info_dict, out_file)
# out_file.close()

### statistics
# 29562 stations
# valid: 28374


29562 28374


In [120]:
# step 3: check china stations missing info -> all valid, filter station_info
count = 0
for station_id in fixed_data:
  if (station_id not in staion_info_dict) :
    count+=1
# print(count)
# 0

china_station_dict = {}
for station_id in fixed_data:
  if (station_id in staion_info_dict) :
    china_station_dict[station_id] = staion_info_dict[station_id]

print(china_station_dict)
out_file = open("station_location.json", "w")
json.dump(china_station_dict, out_file)
out_file.close()


{'450070': {'name': 'HONG KONG INTL', 'lon': 113.915, 'lat': 22.309}, '450110': {'name': 'MACAU INTL', 'lon': 113.592, 'lat': 22.15}, '450320': {'name': 'TA KWU LING', 'lon': 114.15, 'lat': 22.533}, '450350': {'name': 'LAU FAU SHAN', 'lon': 113.983, 'lat': 22.467}, '450390': {'name': 'SHA TIN', 'lon': 114.2, 'lat': 22.4}, '450440': {'name': 'CHEUNG CHAU', 'lon': 114.017, 'lat': 22.2}, '466860': {'name': 'CHIANG KAI SHEK INTL', 'lon': 121.233, 'lat': 25.078}, '466960': {'name': 'SUNGSHAN', 'lon': 121.552, 'lat': 25.069}, '467400': {'name': 'KAOHSIUNG INTL', 'lon': 120.35, 'lat': 22.577}, '467430': {'name': 'TAINAN', 'lon': 120.206, 'lat': 22.95}, '467700': {'name': 'CHING CHUAN KANG AB', 'lon': 120.621, 'lat': 24.265}, '501360': {'name': 'MOHE', 'lon': 122.533, 'lat': 52.967}, '503530': {'name': 'HUMA', 'lon': 126.633, 'lat': 51.733}, '504340': {'name': 'TULIHE', 'lon': 121.7, 'lat': 50.45}, '504680': {'name': 'AIHUI', 'lon': 127.45, 'lat': 50.25}, '505480': {'name': "XIAO'ERGOU", 'lon'

In [5]:
# get calender
import calendar
result = calendar.calendar(2022)
monthrange = calendar.monthrange(2022, 7)
print(monthrange[1])


31


In [ ]:
# step 4: a function to display knn
